# Edit MSHE setup file using mpyshe

To use MShePy, either:

- add the MIKE Zero installation path/bin/x64 directory to the system/user variable PYTHONPATH (preferred) or
- call sys.path.append("MIKE Zero installation path/bin/x64")

This notebook guides you through how to use the MShePy module, a python package that allows you to interact with and run MIKE SHE simulations from python scripts. Documentation for MShePy is found [here](https://docs.mikepoweredbydhi.com/engine_libraries/MShe/MShePyApi/). MShePy is designed specifically for situations where the users wants to modify the current state based on the current state of other variables in the model, for example, calculating the current evaportranspiration based on the modeled soil moisture at a timestep. This can be done with MShePy in two ways, 

- Incorporating python plugins for use in the MIKE SHE user interface
- Running MIKE SHE simulations from python scripts, where the model may contain python plugins.

Here we will walk through how to use the MShePy plugin to run a model from python with a customizable plugin.

### Notices
***NOTE 1: MShePy currently only supports python versions 3.8 - 3.12***

***NOTE 2: MShePy does not allow two models to be initialized in a single python process (see [Limitations](https://docs.mikepoweredbydhi.com/engine_libraries/MShe/MShePyApi/#initializing-a-mike-she-model)). For this reason, you can only run one of the examples in this notebook at a time. Before running the next example, you may need to restart your kernal.***


In [1]:
from datetime import datetime, timedelta
import sys
import os

os.add_dll_directory(r"C:\Program Files (x86)\DHI\MIKE Zero\2025\bin\x64")
sys.path.append(r"C:\Program Files (x86)\DHI\MIKE Zero\2025\bin\x64")
import MShePy as ms # Make sure MShePy is imported before mikeio!
import subprocess as sp
import numpy as np
import mikeio # mikeio 2.6.0 or later is required


MODEL_PATH_SKJERN = r"..\Skjern_Models\Setup\HIP_500m_Skjern_DHI.she"
MODEL_PATH_SIMPLE = r"simple_box\3x3_Box.she"


#### *1. Run the model one timestep at a time, printing min and max precip within the script*
Here we will
- Initialize a MShe model
- Run the model for one timestep
- Print out the min and max precipitation rate at each timestep

In [2]:
ms.wm.initialize(MODEL_PATH_SKJERN)         # Initialize the model
currentTime = ms.wm.currentTime()                        # Find the initial timestep
print(f"Initial time: {currentTime}")

for tstep in range(2):
    nextTime = currentTime + timedelta(days=31)           # Define the next timestep (31 days later) (monthly)
    ms.wm.runToTime(nextTime)                            # Run the model to the next timestep
    currentTime = ms.wm.currentTime()                    # Update the current timestep
    print(f"Current time: {currentTime}")                # Print the current timestep


    # Print min and max precipitation rate
    (dtStart, dtEnd, values) = ms.wm.getValues(ms.paramTypes.P_RATE)
    print(f"Precipitation for period {dtStart} to {dtEnd}")
    print("-------------------------------------------")
    arr = np.array(values[:,:])
    minP = np.nanmin(arr)
    maxP = np.nanmax(arr)
    print(f"Min, Max precipitation rate: {minP} mm/h, {maxP} mm/h")
    print("-------------------------------------------")


ms.wm.terminate(True)                                    # Terminate the simulation

Initial time: 1990-01-02 06:00:00
Current time: 1990-02-02 06:00:00
Precipitation for period 1990-02-01 06:00:00 to 1990-02-02 06:00:00
-------------------------------------------
Min, Max precipitation rate: 0.0 mm/h, 4.2617948281531426e-08 mm/h
-------------------------------------------
Current time: 1990-03-05 06:00:00
Precipitation for period 1990-03-04 06:00:00 to 1990-03-05 06:00:00
-------------------------------------------
Min, Max precipitation rate: 0.0 mm/h, 5.6183878882620775e-08 mm/h
-------------------------------------------


#### *2. Run a simple box model with plugins - demo from https://github.com/DHI/MikeShe/blob/main/SimulationExecution/execute_stepwise_examples.py*
Here we will
- Preprocess the a simple 3x3 box MIKE SHE model and enable plugins
- Create two simple plugins that write variable states to the log file: 1. timesteps, 2. min and max precipitation rate at each timestep
- Run the model from python

In [ ]:
############################################################################################################################################################
# SCRIPT SETUP HERE ALTERED FROM THE DEMO CODE FOUND AT https://github.com/DHI/MikeShe/blob/main/SimulationExecution/execute_stepwise_examples.py
# Alterations:
# - Added to postTimeStep plugin to demonstrate reading model data from within a plugin
# - Added preLeaveSimulator plugin to demonstrate reading model data at the end of the simulation
############################################################################################################################################################

#######################################
# Helper functions
#######################################

def pp(setup):
  """Run the MIKE She preprocessor.
  As no python interface is available use the executable and run it as an external process.
  """
  # Get the directory where MShePy was loaded from and use PP from the same installation
  mz_dir = os.path.dirname(ms.__file__)
  pp_exe = os.path.join(mz_dir, "MShe_Preprocessor.exe")
  sp.run([pp_exe, setup])


def enable_plugin(in_path, out_path):
  """Enable using plugins, set the path to the python interpreter and the
  path to this file for plugins - save as copy.
  """
  # Use the same interpreter that is running this script - but we need the dll, not the exe
  py_dir = os.path.dirname(sys.executable) # path to python.exe
  py_dll = f"python{sys.version_info.major}{sys.version_info.minor}.dll"
  py_path = os.path.join(py_dir, py_dll)

  pfs = mikeio.PfsDocument(in_path, unique_keywords=False)
  pfs.MIKESHE_FLOWMODEL.SimSpec.ModelComp.Plugins = 1
  pfs.MIKESHE_FLOWMODEL.Plugins.PyResolve = 1
  # Set paths - special syntax for pfs paths using '|'
  pfs.MIKESHE_FLOWMODEL.Plugins.PyPath = f"|{py_path}|"
  pfs.MIKESHE_FLOWMODEL.Plugins.PluginFileList.PluginFile_1.FILE_NAME = f"|{ms.__file__}|" # this file
  pfs.write(out_path) # save copy


#######################################
# Plugins
#######################################

def postTimeStep(): # At the end of each timestep
  """A MIKE She plugin function. No technical problem to put it in the same file as the code
  calling the MIKE She engine, however in a larger project it might be cleaner to separate it.
  """
  ms.wm.log(f"Message from plugin: Time step performed, time now: {ms.wm.currentTime()}")

  (dtStart, dtEnd, values) = ms.wm.getValues(ms.paramTypes.P_RATE)
  ms.wm.log(f"Message from plugin: Precipitation data for period {dtStart} to {dtEnd}")

  arr = np.array(values[:,:])
  minP = np.nanmin(arr)
  maxP = np.nanmax(arr)
  ms.wm.log(f"-- Min, Max precipitation rate: {minP} mm/h, {maxP} mm/h")

def preLeaveSimulator():
  """A MIKE She plugin function called after the last timestep, buit before closing MIKE SHE and log files."""
  (dtStart, dtEnd, values) = ms.wm.getValues(ms.paramTypes.P_RATE)
  ms.wm.log(f"Message from plugin: LAST precipitation data for period {dtStart} to {dtEnd}")

  arr = np.array(values[:,:])
  minP = np.nanmin(arr)
  maxP = np.nanmax(arr)
  ms.wm.log(f"-- Min, Max precipitation rate: {minP} mm/h, {maxP} mm/h")


#######################################
# Demo functions
#######################################

def execute_time_steps(setup):
  """Demo: Run the MIKE She water movement engine by taking time steps via the python API."""
  p, e = os.path.splitext(setup)
  setup_plugin = p + "_plugin" + e

  # optional: Enable plugins
  enable_plugin(setup, setup_plugin)
  
  pp(setup_plugin)
  ms.wm.initialize(setup_plugin)
  performed, dt_hours, first_time = ms.wm.performTimeStep()
  if performed:
    ms.wm.log("Initial time step performed!")
    ms.wm.log(f"  Duration: {dt_hours} h")
    ms.wm.log(f"  New time: {first_time}")
    postTimeStep()                                          ## Q: Is this where the plugin is called? Or should it be automatically called by the engine?
  else:
    ms.wm.log("Failed to execute time step via python")
    return
  dt = timedelta(hours=dt_hours * 3.5)
  next_time = first_time + dt
  ms.wm.log(f"Requesting to run to: {next_time}")
  next_time = ms.wm.runToTime(next_time)
  ms.wm.log(f"Actual new date-time after calling runToTime: {next_time}")
  preLeaveSimulator()                                       ## Q: Is this where the plugin is called? Or should it be automatically called by the engine?
  ms.wm.terminate(True)


if __name__ == "__main__":
  execute_time_steps(MODEL_PATH_SIMPLE)


#### To see if this ran properly with plugins enabled, check the log files in the model folder.